## Telegram Scrape Test


In [18]:
import sys
import asyncio
from pathlib import Path

proj = Path().resolve().parent
if str(proj) not in sys.path:
    sys.path.append(str(proj))



In [19]:
from telethon.sync import TelegramClient
from pathlib import Path
from datetime import datetime
import json



api_id = 23969142
api_hash = "63cfef12e419c6f4a413724fccd8dda9"

client = TelegramClient('session', api_id, api_hash)

async def collect_from_channel(channel_username, log_messages):
    await client.start()
    log_messages.append(f"📥 Scraping messages from {channel_username}...")

    all_data = []

    try:
        async for msg in client.iter_messages(channel_username, limit=15):
            message_data = {
                "id": msg.id,
                "date": msg.date.strftime("%Y-%m-%d %H:%M:%S"),
                "text": msg.text if msg.text else None,
                "media": []
            }

            if msg.media:
                media_folder = Path("/home/samrawit/telegram-data-pipeline/data/raw/Image") / datetime.now().strftime("%Y-%m-%d")
                media_folder.mkdir(parents=True, exist_ok=True)

                media_file_path = await msg.download_media(file=media_folder)
                message_data['media'].append(str(media_file_path))
                log_messages.append(f"📸 Downloaded media for message ID {msg.id}")

            all_data.append(message_data)

        log_messages.append(f"✅ Scraped {len(all_data)} messages from {channel_username}.")

        if not all_data:
            log_messages.append(f"⚠️ No messages found in {channel_username} — nothing to save.")
            return

        folder = Path("/home/samrawit/telegram-data-pipeline/data/raw") / datetime.now().strftime("%Y-%m-%d")
        folder.mkdir(parents=True, exist_ok=True)
        out_file = folder / f"{channel_username}.json"

        with open(out_file, "w", encoding="utf-8") as f:
            json.dump(all_data, f, indent=2, ensure_ascii=False)
        log_messages.append(f"✅ Saved to {out_file}")

    except Exception as e:
        log_messages.append(f"❌ Error while scraping or saving from {channel_username}: {e}")

    finally:
        await client.disconnect()

async def main():
    log_messages = []
    await collect_from_channel("lobelia4cosmetics", log_messages)
    await collect_from_channel("tikvahpharma", log_messages)
    
    # Log all messages at once after scraping
    with open('scraper.log', 'a', encoding='utf-8') as log_file:
        for message in log_messages:
            log_file.write(f"{message}\n")

await main()
print("Done scraping and saving!")

Done scraping and saving!
